#### Lab 01 - Content Based Recommender (Film Genre)

In [1]:
import pandas, warnings

warnings.filterwarnings("ignore")

In [2]:
origin = pandas.read_table("movies.csv", sep=",")

origin.iloc[:5]

MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         Genre  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [3]:
origin.tail()

MovieID                                      Title  \
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                Genre  
9737  Action|Animation|Comedy|Fantasy  
9738         Animation|Comedy|Fantasy  
9739                            Drama  
9740                 Action|Animation  
9741                           Comedy

In [4]:
null = origin.isnull().sum()

null.sum()

0

In [5]:
total = origin.shape[0]

total

9742

#### Maintain Genre Column Quality

In [6]:
# Change Genre Column Separator

splita = lambda val : " ".join(val.split("|"))

origin["Genre"] = origin["Genre"].apply(splita)

In [7]:
# Delete Film Without Genre

nolist = "(no genres listed)"

origin = origin[origin["Genre"] != nolist]

origin.iloc[:5]

MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         Genre  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy

Replace `Sci-Fi` to `SciFi`, `Film-Noir` to `Filnoir`

in Our Goal Using `TfidfVectorizer`, Replacing `Sci-Fi` to `SciFi`, `Film-Noir` to `Filnoir`

Others, Resulting `Sci-Fi` to (`Sci` + `Fi`), `Film-Noir` to (`Film` + `Noir`)

In [8]:
# Replace Sci-Fi + Film-Noir

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  origin["Genre"] = origin["Genre"].str.replace(i, t)

origin.iloc[:5]

MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         Genre  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy

In [9]:
# Find List of Genres

ListOfGenre = []

for insect in origin["Genre"].values:

  insect = insect.split(" ")

  for val in insect:

    if val not in ListOfGenre: ListOfGenre.append(val)

len(ListOfGenre)

19

There Are 19 Unique Genre on Our Film Table

In [10]:
# First Five Genres

ListOfGenre[:5]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [11]:
# Converting Genres to TF-IDF (Term Frequency-Inverse Document Frequency) Matrix

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")

metrics = tfidf.fit_transform(origin["Genre"])

metrics.shape

(9708, 19)

In [12]:
tfidf.get_feature_names_out().tolist()[:5]

['action', 'adventure', 'animation', 'children', 'comedy']

In [13]:
len(tfidf.get_feature_names_out().tolist())

19

In [14]:
assert len(tfidf.get_feature_names_out().tolist()) == len(ListOfGenre), "Fail !"

In [15]:
assert metrics.todense().shape == metrics.shape, "Fail !"

In [16]:
metrics.todense()[2]

matrix([[0.        , 0.        , 0.        , 0.        , 0.57070525,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.82115499,
         0.        , 0.        , 0.        , 0.        ]])

In [17]:
origin.iloc[2:3]

MovieID                    Title           Genre
2        3  Grumpier Old Men (1995)  Comedy Romance

In [18]:
metrics.todense()[3]

matrix([[0.        , 0.        , 0.        , 0.        , 0.50488626,
         0.        , 0.        , 0.46621628, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.72645182,
         0.        , 0.        , 0.        , 0.        ]])

In [19]:
origin.iloc[3:4]

MovieID                     Title                 Genre
3        4  Waiting to Exhale (1995)  Comedy Drama Romance

#### Main Model

#### First Time User